# Music Classification into Different Genres

### Amaç: Her birinin 100 farklı örneği olan müzik türlerinin, Keras ile bir modele eğitilmesi ve başarılı sonuçlar alabilmek.

Adımlar:
1. Importing Libs for Preprocessing
2. Uploading & Preprocessing Data
3. Importing Libs for Model Building
4. Building Model
5. Evaluating Model
6. Outcomes
7. Final

## 1. Importing Libs for Preprocessing

Ses verilerimizi işleyebilmek için *librosa* paketimizden faydalanacağız.

In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

C:\Users\arife\anaconda3\envs\tf-gpu\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## 2. Uploading & Preprocessing Data



In [2]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

csvfilepath = 'csvfile/audioDatas.csv'

dgfd

In [3]:
file = open(csvfilepath, 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'genres/{g}'):
        songname = f'genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open(csvfilepath, 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

gözlem

In [4]:
data = pd.read_csv(csvfilepath)
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.au,0.349951,0.130192,1784.416546,2002.657106,3806.418650,0.083066,-113.619385,121.553017,-19.151056,...,8.806787,-3.665802,5.745939,-5.161711,0.750298,-1.688358,-0.409329,-2.298869,1.219947,blues
1,blues.00001.au,0.340945,0.095892,1529.871314,2038.612143,3548.986873,0.056044,-207.581512,123.997147,8.939115,...,5.375942,-2.237833,4.217125,-6.011047,0.932668,-0.717028,0.292333,-0.285717,0.532144,blues
2,blues.00002.au,0.363562,0.175494,1552.637786,1747.382028,3041.089944,0.076301,-90.776344,140.448608,-29.100559,...,5.785763,-8.899733,-1.087024,-9.216774,2.453648,-7.721793,-1.816497,-3.434354,-2.231391,blues
3,blues.00003.au,0.404848,0.141139,1070.110059,1596.244204,2185.061787,0.033309,-199.462006,150.094727,5.649167,...,6.085353,-2.476188,-1.076225,-2.876056,0.776909,-3.320055,0.637263,-0.617507,-3.406940,blues
4,blues.00004.au,0.308598,0.091540,1835.507009,1748.367477,3581.003346,0.101500,-160.291855,126.195763,-35.602501,...,-2.809849,-6.935621,-7.559294,-9.172152,-4.510408,-5.453736,-0.924468,-4.408076,-11.701833,blues


Dropping unneccesary columns

In [5]:
data = data.drop(['filename'],axis=1)
data.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0.349951,0.130192,1784.416546,2002.657106,3806.418650,0.083066,-113.619385,121.553017,-19.151056,42.345768,...,8.806787,-3.665802,5.745939,-5.161711,0.750298,-1.688358,-0.409329,-2.298869,1.219947,blues
1,0.340945,0.095892,1529.871314,2038.612143,3548.986873,0.056044,-207.581512,123.997147,8.939115,35.870762,...,5.375942,-2.237833,4.217125,-6.011047,0.932668,-0.717028,0.292333,-0.285717,0.532144,blues
2,0.363562,0.175494,1552.637786,1747.382028,3041.089944,0.076301,-90.776344,140.448608,-29.100559,31.687344,...,5.785763,-8.899733,-1.087024,-9.216774,2.453648,-7.721793,-1.816497,-3.434354,-2.231391,blues
3,0.404848,0.141139,1070.110059,1596.244204,2185.061787,0.033309,-199.462006,150.094727,5.649167,26.870148,...,6.085353,-2.476188,-1.076225,-2.876056,0.776909,-3.320055,0.637263,-0.617507,-3.406940,blues
4,0.308598,0.091540,1835.507009,1748.367477,3581.003346,0.101500,-160.291855,126.195763,-35.602501,22.151171,...,-2.809849,-6.935621,-7.559294,-9.172152,-4.510408,-5.453736,-0.924468,-4.408076,-11.701833,blues


dfsfs

In [6]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 

fdgfdg

In [7]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## 3. Importing Libs for Model Building

In [8]:
from tensorflow import keras 
from tensorflow.keras import Sequential
from tensorflow.keras import models
from tensorflow.keras import layers

## 4. Building Model

dsfdsgfd model hakkında detay bilgi

In [9]:
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

## 5. Evaluatling
dslkfms

In [10]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               6912      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 48,714
Trainable params: 48,714
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
7/7 [==============================] - 0s 4ms/step - loss: 2.1943 - accuracy: 0.2125
Epoch 2/20
7/7 [==============================] - 0s 3ms/step - loss: 1.8777 - accuracy: 0.3925
Epoch 3/20
7/7 [==============================] - 0s 4ms/step - loss: 1.6235 - accuracy: 0.4400
Epoch 4/20
7/7 [==============================] - 0s 4ms/step - loss: 1.4324 - accuracy: 0.4975
Epoch 5/20
7/7 [==============================] - 0s 5ms/step - loss: 1.2843 - accuracy: 0.5612
Epoch 6/20
7/7 [==============================] - 0s 5ms/step - loss: 1.1717 - accuracy: 0.5938
Epoch 7/20
7/7 [==============================] - 0s 4ms/step - loss: 1.0779 - accuracy: 0.6300
Epoch 8/20
7/7 [==============================] - 0s 3ms/step - loss: 1.0027 - accuracy: 0.6538
Epoch 9/20
7/7 [==============================] - 0s 3ms/step - loss: 0.9380 - accuracy: 0.7013
Epoch 10/20
7/7 [==============================] - 0s 3ms/step - loss: 0.8767 - accuracy: 0.7113
Epoch 11/20
7/7 [======================

dsgdgdg

In [13]:
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

7/7 [==============================] - 0s 3ms/step - loss: 1.0070 - accuracy: 0.6400
test_acc:  0.6399999856948853


fdgfdgdf

In [15]:
predictions = model.predict(X_test)
np.argmax(predictions[2])

0

fdgfdgdfg

In [16]:
maxpredicts = []
for element in predictions:
    temp = np.argmax(element)
    maxpredicts.append(temp)
    
y_pred = np.array(maxpredicts)    

dgfdhdgs

In [17]:
target_names = ['0blues', '1classical', '2country', '3disco', '4hiphop', '5jazz', '6metal', '7pop', '8reggae', '9rock']

dfgfdhdfg

In [18]:
from sklearn.metrics import classification_report
classification_report(y_test, y_pred, target_names=target_names)

'              precision    recall  f1-score   support\n\n      0blues       0.64      0.67      0.65        21\n  1classical       0.86      0.86      0.86        22\n    2country       0.67      0.56      0.61        18\n      3disco       0.59      0.71      0.64        24\n     4hiphop       0.67      0.42      0.51        24\n       5jazz       0.62      0.68      0.65        19\n      6metal       0.72      0.81      0.76        16\n        7pop       0.68      0.77      0.72        22\n     8reggae       0.57      0.71      0.63        17\n       9rock       0.25      0.18      0.21        17\n\n    accuracy                           0.64       200\n   macro avg       0.63      0.64      0.63       200\nweighted avg       0.63      0.64      0.63       200\n'

In [30]:
clsf_report = pd.DataFrame(classification_report(y_true = y_test, y_pred = y_pred, output_dict=True)).transpose()
clsf_report.to_csv('csvfile/Classification Report Name.csv', index= True)

dfgfdgh


In [34]:
clsf_report

,precision,recall,f1-score,support
0,0.636364,0.666667,0.651163,21.00
1,0.863636,0.863636,0.863636,22.00
2,0.666667,0.555556,0.606061,18.00
3,0.586207,0.708333,0.641509,24.00
4,0.666667,0.416667,0.512821,24.00
5,0.619048,0.684211,0.650000,19.00
6,0.722222,0.812500,0.764706,16.00
7,0.680000,0.772727,0.723404,22.00
8,0.571429,0.705882,0.631579,17.00
9,0.250000,0.176471,0.206897,17.00
